In [9]:
import heapq
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import math

In [10]:
def dijkstra_shortest_path(graph, start, target, speeds):
    start = start.lower()
    target = target.lower()
    
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0
    times = {vertex: float('infinity') for vertex in graph}
    times[start] = 0
    previous_vertices = {vertex: None for vertex in graph}

    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, distance in graph[current_vertex].items():
            total_distance = current_distance + distance
            speed = speeds.get((current_vertex, neighbor), None)
            if speed is not None:
                total_time = times[current_vertex] + (distance / speed) + 0.5
            else:
                total_time = float('infinity')

            if total_distance < distances[neighbor]:
                distances[neighbor] = total_distance
                times[neighbor] = total_time
                previous_vertices[neighbor] = current_vertex
                heapq.heappush(priority_queue, (total_distance, neighbor))

    path = []
    current_vertex = target
    while previous_vertices[current_vertex] is not None:
        path.append(current_vertex)
        current_vertex = previous_vertices[current_vertex]
    path.append(start)
    path.reverse()

    return distances[target], times[target], path

# Define AVG speed (605 km/h) Between Airports
speeds = {
    ('med', 'par'): 605, ('med', 'rom'): 605,
    ('par', 'med'): 605, ('par', 'ber'): 605, ('par', 'rom'): 605, ('par', 'lcy'): 605,
    ('rom', 'vie'): 605, ('rom', 'med'): 605, ('rom', 'par'): 605,
    ('lcy', 'dub'): 605, ('lcy', 'sto'): 605, ('lcy', 'par'): 605,
    ('ber', 'msq'): 605, ('ber', 'par'): 605, ('ber', 'sto'): 605, ('ber', 'vie'): 605,
    ('vie', 'gml'): 605, ('vie', 'ber'): 605, ('vie', 'rom'): 605,
    ('sto', 'lcy'): 605, ('sto', 'ber'): 605, ('sto', 'msq'): 605,
    ('msq', 'gml'): 605, ('msq', 'sto'): 605, ('msq', 'ber'): 605,
    ('gml', 'msq'): 605, ('gml', 'vie'): 605,
    ('dub', 'lcy'): 605,
}

graph = {
    'med': {'par': 1048, 'rom': 1347},
    'par': {'lcy': 336, 'ber': 871, 'rom': 1109, 'med': 1048},
    'lcy': {'dub': 451, 'sto': 1431, 'par': 336},
    'ber': {'msq': 988, 'vie': 539, 'par': 871, 'sto': 829},
    'vie': {'ber': 539, 'rom': 774, 'gml': 1027},
    'sto': {'msq': 879, 'ber': 829, 'lcy': 1431},
    'rom': {'vie': 774, 'par': 1109, 'med': 1347},
    'gml': {'msq': 394, 'vie': 1027},
    'dub': {'lcy': 451},
    'msq': {'sto': 879, 'ber': 988, 'gml': 394}
}

In [11]:

def find_path():
    start_airport = entry_start.get().lower()
    target_airport = entry_target.get().lower()

    if start_airport not in graph or target_airport not in graph:
        messagebox.showerror("Error", "Invalid airport code entered.")
        return

    shortest_path_distance, shortest_path_time, shortest_path = dijkstra_shortest_path(graph, start_airport, target_airport, speeds)

    hours = int(shortest_path_time)
    minutes = int((shortest_path_time - hours) * 60)

    result_distance.config(text=f"Distance: {shortest_path_distance} km")
    result_time.config(text=f"Time: {hours} hours and {minutes} minutes")
    result_path.config(text=f"Path: {' -> '.join(shortest_path).upper()}")

    plot_graph(graph, shortest_path)

def plot_graph(graph, path):
    figure.clear()
    ax = figure.add_subplot(111)
    
    for i in range(len(path) - 1):
        start, end = path[i], path[i + 1]
        x = [coords[start.upper()][0], coords[end.upper()][0]]
        y = [coords[start.upper()][1], coords[end.upper()][1]]
        ax.plot(x, y, color='yellow', linewidth=2)

    for airport, (x, y) in coords.items():
        ax.plot(x, y, 'ro')
        ax.text(x, y, airport, fontsize=12, ha='right')

    ax.set_title("Airport Connections")
    ax.axis('equal')
    canvas.draw()

def focus_next_widget(event):
    event.widget.tk_focusNext().focus()
    return("break")

# Calculate coordinates for a circular sequence graph
num_airports = len(graph)
angle_step = 2 * math.pi / num_airports
coords = {}
for i, airport in enumerate(graph.keys()):
    angle = i * angle_step
    x = 10 * math.cos(angle)
    y = 10 * math.sin(angle)
    coords[airport.upper()] = (x, y)

# Create main window
root = tk.Tk()
root.title("Shortest Path Finder")
root.geometry("800x600")
root.configure(bg="black")

input_frame = tk.Frame(root, bg="black")
input_frame.pack(side=tk.LEFT, padx=20, pady=20, fill=tk.Y)

label_start = tk.Label(input_frame, text="Enter Start Destination:", bg="black", fg="yellow", font=("Helvetica", 14))
label_start.grid(row=0, column=0, padx=10, pady=10, sticky=tk.W)
entry_start = tk.Entry(input_frame, font=("Helvetica", 14))
entry_start.grid(row=0, column=1, padx=10, pady=10)
entry_start.bind("<Return>", focus_next_widget)

label_target = tk.Label(input_frame, text="Enter Target Destination:", bg="black", fg="yellow", font=("Helvetica", 14))
label_target.grid(row=1, column=0, padx=10, pady=10, sticky=tk.W)
entry_target = tk.Entry(input_frame, font=("Helvetica", 14))
entry_target.grid(row=1, column=1, padx=10, pady=10)
entry_target.bind("<Return>", focus_next_widget)

find_button = tk.Button(input_frame, text="Find Path", command=find_path, font=("Helvetica", 14), bg="yellow", fg="black")
find_button.grid(row=2, column=0, columnspan=2, pady=20)

result_distance = tk.Label(input_frame, text="Distance: ", bg="black", fg="yellow", font=("Helvetica", 14))
result_distance.grid(row=3, column=0, columnspan=2, pady=5)
result_time = tk.Label(input_frame, text="Time: ", bg="black", fg="yellow", font=("Helvetica", 14))
result_time.grid(row=4, column=0, columnspan=2, pady=5)
result_path = tk.Label(input_frame, text="Path: ", bg="black", fg="yellow", font=("Helvetica", 14))
result_path.grid(row=5, column=0, columnspan=2, pady=5)

choices_frame = tk.Frame(root, bg="black")
choices_frame.pack(side=tk.LEFT, padx=20, pady=20)

airports_info = '''Choose From These Airports: 
MED (Spain)
PAR (France)
STO (Sweden)
LCY (UK)
BER (Germany)
VIE (Austria)
ROM (Italy)
GML (Ukraine)
DUB (Ireland)
MSQ (Belarus)'''
label_airports = tk.Label(choices_frame, text=airports_info, bg="black", fg="yellow", font=("Helvetica", 14), justify=tk.LEFT)
label_airports.pack()

graph_frame = tk.Frame(root, bg="black")
graph_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

figure = Figure(figsize=(8, 6), dpi=100)
canvas = FigureCanvasTkAgg(figure, master=graph_frame)
canvas.draw()
canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

root.mainloop()



In [12]:
def generate_table():
    table_data = []
    
    for start in graph:
        for target in graph:
            if start != target:
                distance, time, path = dijkstra_shortest_path(graph, start, target, speeds)
                hours = int(time)
                minutes = int((time - hours) * 60)
                table_data.append((start.upper(), target.upper(), distance, f"{hours} hours and {minutes} minutes", " -> ".join(path).upper()))
    
    table_window = tk.Tk()
    table_window.title("Shortest Paths Table")
    
    tree = ttk.Treeview(table_window, columns=("Start", "Target", "Distance", "Time", "Path"), show="headings", height=15)
    tree.pack(fill=tk.BOTH, expand=True)
    
    tree.heading("Start", text="Start")
    tree.heading("Target", text="Target")
    tree.heading("Distance", text="Distance (km)")
    tree.heading("Time", text="Time")
    tree.heading("Path", text="Path")
    
    for row in table_data:
        tree.insert("", tk.END, values=row)
    
    tree.column("Start", width=100)
    tree.column("Target", width=100)
    tree.column("Distance", width=100)
    tree.column("Time", width=150)
    tree.column("Path", width=400)
    
    scrollbar = ttk.Scrollbar(table_window, orient=tk.VERTICAL, command=tree.yview)
    tree.configure(yscroll=scrollbar.set)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
    
    table_window.mainloop()

generate_table()
